In [40]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.cluster import KMeans

feature_space = torch.load("/workspaces/dbm25/data/extracted_features.pt")

feature = "hog"

a = []
for element in feature_space:
   a.append(element[feature])

a_array = np.array(a)


In [41]:
k = 10
km = KMeans(n_clusters=k, random_state=0)
km.fit(a_array)

# km.cluster_centers_  → shape (k, 1000)
# km.labels_           → array of length 3006
centroids = km.cluster_centers_
labels = km.labels_

print(centroids.shape)
print(labels.shape)

(10, 900)
(3006,)


In [ ]:
point_similarities = []
for idx, point in enumerate(a_array):
    # compute the distance of the feature from each centroid with cosine similarity
    similarities = []
    for idy, centroid in enumerate(centroids):
        sim = np.dot(point, centroid) / (np.linalg.norm(point) * np.linalg.norm(centroid))  # cosine similarity
        similarities.append(np.array([sim, idy]))
    similarities = np.array(similarities)
    sorted_similarities = similarities[similarities[:, 0].argsort()[::-1]]
    point_similarities.append(sorted_similarities)
    
point_distances = np.array(point_similarities)
# Has shape (n_samples, similarity, centroid_id)
print(point_distances.shape)
print(point_distances[0])

(3006, 10, 2)
[[0.88963338 2.        ]
 [0.85781755 8.        ]
 [0.81071696 1.        ]
 [0.79215841 0.        ]
 [0.76504046 6.        ]
 [0.76192477 9.        ]
 [0.7598861  4.        ]
 [0.68426075 3.        ]
 [0.66367538 7.        ]
 [0.6461779  5.        ]]


In [59]:
# Select m<k most important similarities and keep only that
# So for m = 5 and k = 10 we would go from (n_samples, 10, 2) to (n_samples, 5, 2)
m = 10
test_subject = 0
if m <= k:
    reduced_point_distances = point_distances[:, :m, :]
    print(reduced_point_distances.shape)
    print(reduced_point_distances[test_subject])
else:
    print("m must be smaller or equal to k")

(3006, 10, 2)
[[0.88963338 2.        ]
 [0.85781755 8.        ]
 [0.81071696 1.        ]
 [0.79215841 0.        ]
 [0.76504046 6.        ]
 [0.76192477 9.        ]
 [0.7598861  4.        ]
 [0.68426075 3.        ]
 [0.66367538 7.        ]
 [0.6461779  5.        ]]


In [60]:
# Predict point test_subject with kmeans and get it's cluster
# This should return the same cluster as the first element of point_distances
predicted_label = km.predict(a_array[test_subject].reshape(1, -1))
print(predicted_label)

[2]


In [ ]:
# TODO reconstruct the dict with image_id, class, reduced_similarities